In [ ]:
import logging

# Set logging level to WARNING to suppress INFO messages
logging.getLogger("py4j").setLevel(logging.WARNING)

In [ ]:
import yaml
from helper.functions import (
    get_enhanced_retriever,
    EnhancedRetrievalQA
)

# Load configuration
with open("./config/training_config.yaml", "r") as file:
    config = yaml.safe_load(file)

catalog_name = config["data"]["catalog_name"]
schema_name = config["data"]["schema_name"]

vs_endpoint_prefix = "vs_endpoint_"
vs_endpoint_name = vs_endpoint_prefix + "poc"

# Define both vector store indices
content_vs_index = f"{catalog_name}.{schema_name}.pdf_content_vs_index"

# Ensure all required fields are present in the configuration
if "endpoint" not in config:
    config["endpoint"] = {"disable_notice": True}

print("✅ Configuration loaded successfully")
print(f"📊 Catalog: {catalog_name}")
print(f"📊 Schema: {schema_name}")
print(f"🔍 Vector Search Endpoint: {vs_endpoint_name}")
print(f"📖 Content Index: {content_vs_index}")

In [ ]:
from langchain.chat_models import ChatDatabricks, ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
openai_api_key = config["auth"].get("openai_api_key", "")
embeddings_model_endpoint = "text-embedding-3-large"  # OpenAI embedding model
embedding_model = OpenAIEmbeddings(model=embeddings_model_endpoint, openai_api_key=openai_api_key)

# Databricks Llama-4 Maverick
llm_endpoint_name = "databricks-llama-4-maverick"
chat_model_dbx = ChatDatabricks(
    endpoint=llm_endpoint_name, 
    max_tokens=1000, 
    temperature=0.1, 
    disable_notice=True
 )
# OpenAI GPT-4.1-mini
chat_model_openai = ChatOpenAI(
    model_name="gpt-4-1106-preview",  # GPT-4.1-mini
    openai_api_key=openai_api_key,
    max_tokens=1000,
    temperature=0.1
)

In [ ]:
def generate_answers(question_list, prompt_template, chat_model, model_name):
    print(f"\n🔧 Setting up Enhanced RAG System using {model_name}...")
    try:
        enhanced_retriever = get_enhanced_retriever(
            vs_endpoint_name=vs_endpoint_name,
            vs_index_fullname=content_vs_index,
            k=15
        )
        enhanced_qa = EnhancedRetrievalQA(
            llm=chat_model,
            enhanced_retriever=enhanced_retriever,
            custom_prompt_template=prompt_template
        )
        print(f"✅ Enhanced RAG System initialized with {model_name}!")
        print(f"🧪 Testing Enhanced RAG System with {model_name}:")
        print("=" * 80)
        for i, question in enumerate(question_list, 1):
            print(f"\n📝 Question {i}: {question}")
            print("-" * 60)
            try:
                result = enhanced_qa(question)
                print(f"Answer ({model_name}):")
                print(f"{result['result']}")
            except Exception as e:
                print(f"⚠️  Question {i} skipped due to authentication requirements")
                print(f"   Error: {str(e)[:100]}...")
                print("=" * 80)
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        raise

In [ ]:
from langchain.prompts import PromptTemplate

custom_prompt_template = PromptTemplate()

In [ ]:
test_questions = [
    "What is the total assets for the bank and group in 2024?",
    "What is the total loans and advances to customer for the bank and group in 2024?"
    "What is the 'Gain on partial disposal of subsidiary' for the group in 2024?",
    "What is the Total equity for the bank in 2023?",
    "What is the Net cash generated from operating activities for the bank in 2023?",
    "What is the number of branches in Ghana?",
    "What are the Total impairment allowances on loans per IFRS for the bank in December 2024 from the statement of prudential adjustments?",
    "Who is the Managing Director/CEO of access bank?",
    "What are the Core values of the bank?"
    "What are the audit fees paid by the Bank to KPMG Professional Services for year the 2024 statutory audit?",
    "List the key achievements done in Ghana subsidiary branches"
    ]

In [ ]:
generate_answers(test_questions, custom_prompt_template, chat_model_openai, "GPT-4.1-mini (OpenAI)")